# 08: Mesh Networking

This notebook explores mesh networking concepts with a focus on Meshtastic-style protocols.

## Learning Objectives
- Understand mesh network topology and routing
- Explore flooding vs routing protocols
- Analyze CSMA/CA channel access
- Simulate multi-hop message delivery

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## Mesh Network Basics

In a **mesh network**, nodes communicate directly with each other:
- No central infrastructure required
- Self-healing: routes around failures
- Range extension through multi-hop

### Meshtastic
Open-source mesh networking using LoRa:
- ~10km range per hop (LoRa)
- Managed flood routing
- 40,000+ nodes worldwide

In [ ]:
class MeshNode:
    """Simple mesh network node."""
    
    def __init__(self, node_id, x, y, tx_range=100):
        self.node_id = node_id
        self.x = x
        self.y = y
        self.tx_range = tx_range
        self.seen_packets = set()  # For duplicate detection
        self.neighbors = []  # Discovered neighbors
        self.message_queue = deque()
    
    def distance_to(self, other):
        return np.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)
    
    def can_reach(self, other):
        return self.distance_to(other) <= self.tx_range
    
    def __repr__(self):
        return f"Node({self.node_id})"

In [ ]:
# Create a random mesh network
np.random.seed(42)

num_nodes = 20
area_size = 400
tx_range = 120

nodes = [
    MeshNode(i, 
             np.random.uniform(0, area_size),
             np.random.uniform(0, area_size),
             tx_range)
    for i in range(num_nodes)
]

# Discover neighbors
for node in nodes:
    node.neighbors = [n for n in nodes if n != node and node.can_reach(n)]

In [ ]:
# Visualize the network
fig, ax = plt.subplots(figsize=(10, 10))

# Draw connections
for node in nodes:
    for neighbor in node.neighbors:
        ax.plot([node.x, neighbor.x], [node.y, neighbor.y], 
                'b-', alpha=0.2, linewidth=0.5)

# Draw nodes
xs = [n.x for n in nodes]
ys = [n.y for n in nodes]
ax.scatter(xs, ys, s=100, c='blue', zorder=5)

# Label nodes
for node in nodes:
    ax.annotate(str(node.node_id), (node.x + 5, node.y + 5))

ax.set_xlim(-20, area_size + 20)
ax.set_ylim(-20, area_size + 20)
ax.set_aspect('equal')
ax.set_title(f'Mesh Network ({num_nodes} nodes, range={tx_range}m)')
ax.set_xlabel('X (meters)')
ax.set_ylabel('Y (meters)')
plt.show()

# Network statistics
avg_neighbors = np.mean([len(n.neighbors) for n in nodes])
print(f"Average neighbors per node: {avg_neighbors:.1f}")

## Flooding Protocol

**Simple flooding**: Broadcast to all neighbors, they re-broadcast

Advantages:
- Simple implementation
- No routing tables needed
- Robust to topology changes

Disadvantages:
- High channel usage
- Broadcast storms possible

In [ ]:
def simulate_flood(nodes, source_id, max_hops=7):
    """Simulate flooding from source node."""
    # Reset seen packets
    for node in nodes:
        node.seen_packets = set()
    
    packet_id = random.randint(0, 2**32)
    reached = {source_id: 0}  # node_id -> hop count
    transmissions = 0
    
    # Initial broadcast
    current_wave = [(nodes[source_id], 0)]  # (node, hop_count)
    nodes[source_id].seen_packets.add(packet_id)
    
    while current_wave:
        next_wave = []
        
        for node, hops in current_wave:
            if hops >= max_hops:
                continue
                
            transmissions += 1
            
            # Broadcast to neighbors
            for neighbor in node.neighbors:
                if packet_id not in neighbor.seen_packets:
                    neighbor.seen_packets.add(packet_id)
                    reached[neighbor.node_id] = hops + 1
                    next_wave.append((neighbor, hops + 1))
        
        current_wave = next_wave
    
    return reached, transmissions

# Test flooding from node 0
reached, tx_count = simulate_flood(nodes, 0)
print(f"Message reached {len(reached)}/{num_nodes} nodes")
print(f"Total transmissions: {tx_count}")
print(f"Redundancy factor: {tx_count/len(reached):.1f}x")

In [ ]:
# Visualize flood propagation
fig, ax = plt.subplots(figsize=(10, 10))

# Color nodes by hop count
max_hops = max(reached.values())
colors = plt.cm.YlOrRd(np.linspace(0.2, 1, max_hops + 1))

for node in nodes:
    if node.node_id in reached:
        hops = reached[node.node_id]
        color = colors[hops]
        ax.scatter(node.x, node.y, s=200, c=[color], zorder=5)
        ax.annotate(f"{node.node_id}\n({hops})", (node.x + 5, node.y + 5), fontsize=8)
    else:
        ax.scatter(node.x, node.y, s=100, c='gray', zorder=5, alpha=0.5)

# Mark source
ax.scatter(nodes[0].x, nodes[0].y, s=300, c='green', marker='*', zorder=6, label='Source')

ax.set_xlim(-20, area_size + 20)
ax.set_ylim(-20, area_size + 20)
ax.set_aspect('equal')
ax.set_title('Flood Propagation (color = hop count)')
ax.legend()
plt.show()

## CSMA/CA Channel Access

**Carrier Sense Multiple Access with Collision Avoidance**:

1. Listen before transmit
2. If channel busy, wait random backoff
3. Transmit if channel free
4. Repeat if needed

In [ ]:
def simulate_csma(num_nodes, num_packets, slot_time=1, max_backoff=16):
    """Simulate CSMA/CA channel access."""
    channel_busy_until = 0
    successful = 0
    collisions = 0
    total_delay = 0
    
    # Each node has packets to send
    queues = [list(range(num_packets)) for _ in range(num_nodes)]
    backoffs = [0] * num_nodes
    current_time = 0
    
    while any(queues):
        # Find nodes ready to transmit (backoff expired)
        ready = [i for i in range(num_nodes) if queues[i] and backoffs[i] <= current_time]
        
        if len(ready) == 0:
            # Advance time to next backoff expiry
            pending = [backoffs[i] for i in range(num_nodes) if queues[i]]
            if pending:
                current_time = min(pending)
            continue
        
        if channel_busy_until > current_time:
            # Channel busy, set new backoff for all ready nodes
            for i in ready:
                backoffs[i] = channel_busy_until + random.randint(1, max_backoff) * slot_time
            current_time = channel_busy_until
            continue
        
        if len(ready) == 1:
            # Successful transmission
            node = ready[0]
            pkt = queues[node].pop(0)
            total_delay += current_time - pkt * slot_time
            successful += 1
            channel_busy_until = current_time + slot_time
        else:
            # Collision!
            collisions += 1
            channel_busy_until = current_time + slot_time
            for i in ready:
                backoffs[i] = channel_busy_until + random.randint(1, max_backoff * 2) * slot_time
        
        current_time += slot_time
    
    return successful, collisions, total_delay / successful if successful else 0

# Test CSMA with varying load
results = []
for n in range(2, 21, 2):
    success, coll, delay = simulate_csma(n, 10)
    throughput = success / (success + coll) if (success + coll) > 0 else 0
    results.append((n, throughput, delay))
    print(f"Nodes: {n:2d}, Throughput: {throughput:.1%}, Avg Delay: {delay:.1f} slots")

In [ ]:
# Plot CSMA performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

nodes_list = [r[0] for r in results]
throughputs = [r[1] for r in results]
delays = [r[2] for r in results]

axes[0].plot(nodes_list, throughputs, 'o-')
axes[0].set_xlabel('Number of Nodes')
axes[0].set_ylabel('Throughput')
axes[0].set_title('CSMA/CA Throughput vs Load')
axes[0].set_ylim(0, 1.1)

axes[1].plot(nodes_list, delays, 's-', color='orange')
axes[1].set_xlabel('Number of Nodes')
axes[1].set_ylabel('Average Delay (slots)')
axes[1].set_title('CSMA/CA Delay vs Load')

plt.tight_layout()
plt.show()

## Mesh Network Metrics

Key performance indicators:
- **Packet Delivery Ratio (PDR)**: % of packets delivered
- **End-to-End Delay**: Time from source to destination
- **Hop Count**: Number of hops to destination
- **Network Lifetime**: Time until first node fails (battery)

In [ ]:
# Analyze network connectivity
def find_path(nodes, source_id, dest_id):
    """Find shortest path using BFS."""
    if source_id == dest_id:
        return [source_id]
    
    visited = {source_id}
    queue = deque([(source_id, [source_id])])
    
    while queue:
        current, path = queue.popleft()
        
        for neighbor in nodes[current].neighbors:
            if neighbor.node_id == dest_id:
                return path + [dest_id]
            
            if neighbor.node_id not in visited:
                visited.add(neighbor.node_id)
                queue.append((neighbor.node_id, path + [neighbor.node_id]))
    
    return None  # No path found

# Calculate all-pairs shortest paths
hop_counts = []
unreachable = 0

for i in range(num_nodes):
    for j in range(i+1, num_nodes):
        path = find_path(nodes, i, j)
        if path:
            hop_counts.append(len(path) - 1)
        else:
            unreachable += 1

print(f"Network Statistics:")
print(f"  Reachable pairs: {len(hop_counts)}/{num_nodes*(num_nodes-1)//2}")
print(f"  Average path length: {np.mean(hop_counts):.2f} hops")
print(f"  Maximum path length: {max(hop_counts)} hops")
print(f"  Network diameter: {max(hop_counts)} hops")

In [ ]:
# Plot hop count distribution
plt.figure(figsize=(8, 5))
plt.hist(hop_counts, bins=range(1, max(hop_counts)+2), align='left', edgecolor='black')
plt.xlabel('Hop Count')
plt.ylabel('Number of Node Pairs')
plt.title('Distribution of Path Lengths')
plt.xticks(range(1, max(hop_counts)+1))
plt.show()

## Meshtastic Parameters

| Parameter | Short Range | Medium Range | Long Range |
|-----------|-------------|--------------|------------|
| SF | 7 | 9 | 12 |
| BW | 250 kHz | 125 kHz | 125 kHz |
| Range | ~3 km | ~8 km | ~15+ km |
| Data Rate | 5.5 kbps | 0.98 kbps | 0.29 kbps |
| Airtime | Short | Medium | Long |

## Exercises

1. **Network density**: What happens to connectivity as you reduce tx_range from 120m to 60m?

2. **Hop limit**: How does changing max_hops from 7 to 3 affect coverage vs channel usage?

3. **Hidden node**: Simulate the hidden node problem where A can reach B, B can reach C, but A cannot reach C. What happens when A and C transmit simultaneously?

In [ ]:
# Your code here
